In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import os

# 파일 경로에서 사용자 코드 추출
file_path = '../data/user_code_total/user_code_006.csv'
user_code = os.path.basename(file_path).split('.')[0]  # 'user_code_006'


# 데이터 불러오기 및 전처리
df = pd.read_csv(file_path) # 학습데이터셋의 경로
df['RegisterDate'] = pd.to_datetime(df['RegisterDate'])
df.sort_values(by='RegisterDate', inplace=True)
df.fillna(method='ffill', inplace=True)


# 심박수 데이터 추출 및 스케일링
heartbeat_data = df['Heartbeat'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
heartbeat_data_scaled = scaler.fit_transform(heartbeat_data)

# PyTorch 텐서로 변환
X = torch.Tensor(heartbeat_data_scaled).unsqueeze(2)

# 데이터셋을 학습 및 검증 셋으로 분리
X_train, X_valid = train_test_split(X, test_size=0.2, random_state=42)

# 데이터 로더 생성
train_dataset = TensorDataset(X_train, X_train)  # 오토인코더의 경우 입력이 타겟입니다
valid_dataset = TensorDataset(X_valid, X_valid)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=64, shuffle=False)

# 오토인코더 모델 정의
class Autoencoder(nn.Module):
    # ... (클래스 정의는 이전과 동일하게 유지)
        def __init__(self, input_dim, hidden_dim, num_layers):
            super(Autoencoder, self).__init__()
            self.encoder = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
            self.decoder = nn.LSTM(hidden_dim, input_dim, num_layers, batch_first=True)
            self.linear = nn.Linear(input_dim, input_dim)

        def forward(self, x):
            _, (hn, _) = self.encoder(x)
            hn = hn[-1].unsqueeze(0)
            repeated_hn = hn.repeat(1, x.size(1), 1)
            decoded, _ = self.decoder(repeated_hn)
            decoded = decoded.contiguous().view(-1, x.size(2))
            decoded = self.linear(decoded)
            return decoded.view(x.size(0), x.size(1), -1)

# 모델 초기화
model = Autoencoder(input_dim=1, hidden_dim=50, num_layers=2)

# 손실 함수와 옵티마이저
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 과정
epochs = 100
best_loss = float('inf')
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # 검증 과정
    model.eval()
    valid_loss = 0.0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            valid_loss += loss.item()

    # 평균 손실 계산
    train_loss = train_loss / len(train_loader)
    valid_loss = valid_loss / len(valid_loader)
    
    print(f'Epoch {epoch+1} - Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}')
    
    # 최고 성능 모델 저장
    if valid_loss < best_loss:
        best_loss = valid_loss
        # torch.save(model.state_dict(), 'best_model_.pth')
        model_save_path = f'best_model_{user_code}.pth'  # 사용자 코드를 파일 이름에 포함
        torch.save(model.state_dict(), model_save_path)

print('Training complete.')


C:\Users\user\AppData\Local\Temp\ipykernel_2300\934902888.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


Epoch 1 - Train Loss: 1.7111, Valid Loss: 1.2022
Epoch 2 - Train Loss: 0.8865, Valid Loss: 0.5916
Epoch 3 - Train Loss: 0.2593, Valid Loss: 0.0280
Epoch 4 - Train Loss: 0.0274, Valid Loss: 0.0256
Epoch 5 - Train Loss: 0.0251, Valid Loss: 0.0237
Epoch 6 - Train Loss: 0.0233, Valid Loss: 0.0223
Epoch 7 - Train Loss: 0.0221, Valid Loss: 0.0212
Epoch 8 - Train Loss: 0.0187, Valid Loss: 0.0105
Epoch 9 - Train Loss: 0.0088, Valid Loss: 0.0068
Epoch 10 - Train Loss: 0.0058, Valid Loss: 0.0045
Epoch 11 - Train Loss: 0.0038, Valid Loss: 0.0029
Epoch 12 - Train Loss: 0.0025, Valid Loss: 0.0020
Epoch 13 - Train Loss: 0.0017, Valid Loss: 0.0014
Epoch 14 - Train Loss: 0.0013, Valid Loss: 0.0011
Epoch 15 - Train Loss: 0.0010, Valid Loss: 0.0008
Epoch 16 - Train Loss: 0.0008, Valid Loss: 0.0007
Epoch 17 - Train Loss: 0.0006, Valid Loss: 0.0006
Epoch 18 - Train Loss: 0.0005, Valid Loss: 0.0005
Epoch 19 - Train Loss: 0.0004, Valid Loss: 0.0004
Epoch 20 - Train Loss: 0.0004, Valid Loss: 0.0004
Epoch 21 